In [128]:
#%save Lecture_data_bresil.py 125

In [1]:
from Lecture_csv import Lecture_csv

class Lecture_data_bresil () :
    
    def __init__ (self, path = './') :
        
        self.path = path
        
        self.dico_csv = {
            'customer' : 'olist_customers_dataset.csv',
            'geolocation' : 'olist_geolocation_dataset.csv',
            'orders_items' : 'olist_order_items_dataset.csv',
            'order_payments' : 'olist_order_payments_dataset.csv',
            'order_reviews' : 'olist_order_reviews_dataset.csv',
            'orders' : 'olist_orders_dataset.csv',
            'products' : 'olist_products_dataset.csv',
            'sellers' : 'olist_sellers_dataset.csv',
            'product_category_name_translation' : 'product_category_name_translation.csv',
            }
        
        self.nombre_erreur = 0
        
    def get_dico (self, nameFile, nom_zone) :
        
        nameFile_reel = self.dico_csv [nameFile]
        
        L = Lecture_csv (self.path, nameFile_reel)
        dico = {}
        
        for row in L.read_iterator ():
            dico [row[nom_zone] ] = row
            continue
        return dico
    
    def add (self,
             dico,
             
             nom_lien1, dico1,
             nom_lien2, dico2,
             
             liste_ajout,
             row_erreur,
             is_del_bad_ID = False,
            
            ) :
        
        liste_bad_ID = []
        for ID, data_dico_final in dico.items() :
            
            isErreur = False
            try :
                valeur_lien = data_dico_final [nom_lien1]
                # la cle de dico1 == zone qui est lie à nom_lien1 dans  enreg dico (par ID customer)
                data_dico = dico1 [valeur_lien] 
            except:
                self.nombre_erreur += 1
                isErreur = True
                pass
        
                
            if isErreur and not nom_lien2 is None:
                try :
                    valeur_lien = data_dico_final [nom_lien2]
                    data_dico = dico1 [valeur_lien]
                except:
                    self.nombre_erreur += 1
                    if is_del_bad_ID :
                        liste_bad_ID.append (ID)
                        continue
                    
                    dico [ID] = row_erreur
                    
                    continue
                      
               
            for nom_zone in liste_ajout  :
                data = data_dico [nom_zone]
                data_dico_final [nom_zone] = data
                
                continue
            dico [ID] = data_dico_final
            continue
        for ID in liste_bad_ID :
            del dico [ID]
                
        return dico
    
    
    def join_consumer_geolocalization (self, ) :
        """
        on realise la jointure par la geolisation
        ou
        par le nom de ville (deuxieme choix)
                
        """
        
        nameFile = 'customer'
        nom_zone = 'customer_id'
        dico = self.get_dico (nameFile, nom_zone)
        
        nameFile = 'geolocation'
    
        nom_zone = 'geolocation_zip_code_prefix'
        dico1 = self.get_dico (nameFile, nom_zone)
        nom_lien1 = 'customer_zip_code_prefix'

        nom_zone = 'geolocation_city'
        dico2 = self.get_dico (nameFile, nom_zone)
        nom_lien2 = 'customer_city'


        liste_ajout = ['geolocation_lat', 'geolocation_lng',
                       'geolocation_state',
                      ]


        row_erreur = {}
        for nom_zone in liste_ajout :
            row_erreur [nom_zone] = ''

        
        dico_resultat  = self.add (  dico,
                                     nom_lien1, dico1,
                                     nom_lien2, dico2,
                                     liste_ajout,
                                     row_erreur,
                                     is_del_bad_ID = True,

                                  )
        
        return dico_resultat
    
    def join_customers_orders (self,dico_resultat) :
        
        nom_zone = 'customer_id'
        dico = dico_resultat 

        nameFile = 'orders'
        nom_zone = 'customer_id' # cle d' acces
        dico1 = self.get_dico (nameFile, nom_zone)
        nom_lien1 = 'customer_id'


        '''
        nom_zone = 'geolocation_city'
        dico2 = self.get_dico (nameFile, nom_zone)
        nom_lien2 = 'customer_city'
        '''
        nom_lien2 = None
        dico2 = {}



        liste_ajout = ['order_id',
                         'order_status',
                         'order_purchase_timestamp',
                         'order_approved_at',
                         'order_delivered_carrier_date',
                         'order_delivered_customer_date',
                         'order_estimated_delivery_date']


        row_erreur = {}
        for nom_zone in liste_ajout :
            row_erreur [nom_zone] = ''

        dico_resultat  = self.add (  dico,
                                         nom_lien1, dico1,
                                         nom_lien2, dico2,
                                         liste_ajout,
                                         row_erreur,
                                         is_del_bad_ID = True,

                                      )
        
        return dico_resultat
    
    def join_customers_orders_items (self, dico_resultat) :
        
        dico = dico_resultat 
        nameFile = 'orders_items'

        nom_zone = 'order_id' # cle d' acces du dico1
        dico1 = self.get_dico (nameFile, nom_zone)
        nom_lien1 = 'order_id' # valeur dans enreg dico


        '''
        nom_zone = 'geolocation_city'
        dico2 = self.get_dico (nameFile, nom_zone)
        nom_lien2 = 'customer_city'
        '''
        nom_lien2 = None
        dico2 = {}



        liste_ajout = [  'product_id',
                         'seller_id',
                         'shipping_limit_date',
                         'price',
                         'freight_value']


        row_erreur = {}
        for nom_zone in liste_ajout :
            row_erreur [nom_zone] = ''

        dico_resultat  = self.add (  dico,
                                     nom_lien1, dico1,
                                     nom_lien2, dico2,
                                     liste_ajout,
                                     row_erreur,
                                     is_del_bad_ID = True,

                                      )
        
        return dico_resultat
    
    def join_customers_products (self, dico_resultat):
        
        dico = dico_resultat 
        nameFile = 'products'

        nom_zone = 'product_id' 
        # cle d' acces du dico1
        dico1 = self.get_dico (nameFile, nom_zone)
        nom_lien1 = 'product_id' # valeur dans enreg dico

        nom_lien2 = None
        dico2 = {}



        liste_ajout = [
                        'product_category_name',
                        'product_name_lenght',
                        'product_description_lenght',
                        'product_photos_qty',
                        'product_weight_g',
                        'product_length_cm',
                        'product_height_cm',
                        'product_width_cm']


        row_erreur = {}
        for nom_zone in liste_ajout :
            row_erreur [nom_zone] = ''

        dico_resultat  = self.add (  dico,
                                     nom_lien1, dico1,
                                     nom_lien2, dico2,
                                     liste_ajout,
                                     row_erreur,
                                     is_del_bad_ID = True,

                                      )
        
        return dico_resultat
    
    def join_customers_products_translation (self, dico_resultat):
        
        dico = dico_resultat 
        nameFile = 'product_category_name_translation'

        nom_zone = '\ufeffproduct_category_name'
        # cle d' acces du dico1
        dico1 = self.get_dico (nameFile, nom_zone)

        nom_lien1 = 'product_category_name' # valeur dans enreg dico

        nom_lien2 = None
        dico2 = {}



        liste_ajout = [ 'product_category_name_english']


        row_erreur = {}
        for nom_zone in liste_ajout :
            row_erreur [nom_zone] = ''

        dico_resultat  = self.add (  dico,
                                     nom_lien1, dico1,
                                     nom_lien2, dico2,
                                     liste_ajout,
                                     row_erreur,
                                     is_del_bad_ID = True,

                                      )
        
        return dico_resultat
    
    def join_customers_sellers (self, dico_resultat):
        
        dico = dico_resultat 
        nameFile = 'sellers'

        nom_zone = 'seller_id'
        # cle d' acces du dico1
        dico1 = self.get_dico (nameFile, nom_zone)
        
        nom_lien1 = 'seller_id' # valeur dans enreg dico

        nom_lien2 = None
        dico2 = {}



        liste_ajout = [ 'seller_city', 'seller_state']

        row_erreur = {}
        for nom_zone in liste_ajout :
            row_erreur [nom_zone] = ''

        dico_resultat  = self.add (  dico,
                                     nom_lien1, dico1,
                                     nom_lien2, dico2,
                                     liste_ajout,
                                     row_erreur,
                                     is_del_bad_ID = True,

                                      )
        
        return dico_resultat
    
    
    def join_customers_payments (self, dico_resultat):
        
        dico = dico_resultat 
        nameFile = 'order_payments'

        nom_zone = 'order_id'
        # cle d' acces du dico1
        dico1 = self.get_dico (nameFile, nom_zone)

        nom_lien1 = 'order_id' # valeur dans enreg dico

        nom_lien2 = None
        dico2 = {}



        liste_ajout = [
                         'payment_sequential',
                         'payment_type',
                         'payment_installments',
                         'payment_value']


        row_erreur = {}
        for nom_zone in liste_ajout :
            row_erreur [nom_zone] = ''

        dico_resultat  = self.add (  dico,
                                     nom_lien1, dico1,
                                     nom_lien2, dico2,
                                     liste_ajout,
                                     row_erreur,
                                     is_del_bad_ID = True,

                                      )
        
        return dico_resultat
    
    def join_customers_reviews (self, dico_resultat):
        
        dico = dico_resultat 
        nameFile = 'order_reviews'

        nom_zone = 'order_id'
        # cle d' acces du dico1
        dico1 = self.get_dico (nameFile, nom_zone)

        nom_lien1 = 'order_id' # valeur dans enreg dico

        nom_lien2 = None
        dico2 = {}



        liste_ajout = [
                         'review_score',
                         'review_comment_title',
                         'review_comment_message',
                         'review_creation_date',
                         'review_answer_timestamp']


        row_erreur = {}
        for nom_zone in liste_ajout :
            row_erreur [nom_zone] = ''

        dico_resultat  = self.add (  dico,
                                     nom_lien1, dico1,
                                     nom_lien2, dico2,
                                     liste_ajout,
                                     row_erreur,
                                     is_del_bad_ID = True,

                                      )
        
        
        return dico_resultat
    
    
    def dico_customers (self,) :
        
        
        nameFile = 'customer'
        nom_zone = 'customer_id'
        dico = self.get_dico (nameFile, nom_zone)

        


        dico_resultat  = self.join_consumer_geolocalization ()


        assert len (dico_resultat) == 99163


        dico_resultat  = self.join_customers_orders (dico_resultat)

        

        # customers orders_items


        dico_resultat = self.join_customers_orders_items (dico_resultat)

        

        # customers customer lien à product

        dico_resultat = self.join_customers_products ( dico_resultat)
        # customers customer lien à product_category_name (traduction english)
            

        dico_resultat  = self.join_customers_products_translation (dico_resultat)

        # consumers à sellers
        dico_resultat = self.join_customers_sellers ( dico_resultat)
        
        #  customer  payment

        dico_resultat = self.join_customers_payments (dico_resultat)

        # customer review
               
        dico_resultat = self.join_customers_reviews (dico_resultat)
                
        return dico_resultat
        
       

In [2]:
#%save test_lecture_data_bresil.py 3

In [2]:
import time
import pandas as pd
from Lecture_data_bresil import Lecture_data_bresil

def test_lecture_data_bresil ():
    
    L = Lecture_data_bresil ()
       
    dico_resultat = L.dico_customers ()
    
    assert L.nombre_erreur == 2765
    
    assert len(dico_resultat) == 99163
    
    
    
    
        
    
    
if __name__ == '__main__' :
    t = time.time()
    test_lecture_data_bresil ()
    print ('fin test_lecture_data_bresil en ', time.time () - t)    

fin test_lecture_data_bresil en  6.497064590454102


In [1]:
import pandas as pd


df = pd.read_csv('olist_customers_dataset.csv', delimiter = ',')
df

,customer_id,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state
0,06b8999e2fba1a1fbc88172c00ba8bc7,861eff4711a542e4b93843c6dd7febb0,14409,franca,SP
1,18955e83d337fd6b2def6b18a428ac77,290c77bc529b7ac935b93aa66c333dc3,9790,sao bernardo do campo,SP
2,4e7b3e00288586ebd08712fdd0374a03,060e732b5b29e8181a18229c7b0b2b5e,1151,sao paulo,SP
3,b2b6027bc5c5109e529d4dc6358b12c3,259dac757896d24d7702b9acbbff3f3c,8775,mogi das cruzes,SP
4,4f2d8ab171c80ec8364f7c12e35b23ad,345ecd01c38d18a9036ed96c73b8d066,13056,campinas,SP
...,...,...,...,...,...
99436,17ddf5dd5d51696bb3d7c6291687be6f,1a29b476fee25c95fbafc67c5ac95cf8,3937,sao paulo,SP
99437,e7b71a9017aa05c9a7fd292d714858e8,d52a67c98be1cf6a5c84435bd38d095d,6764,taboao da serra,SP
99438,5e28dfe12db7fb50a4b2f691faecea5e,e9f50caf99f032f0bf3c55141f019d99,60115,fortaleza,CE
99439,56b18e2166679b8a959d72dd06da27f9,73c2643a0a458b49f58cea58833b192e,92120,canoas,RS


In [16]:
x = df.customer_zip_code_prefix.unique ().tolist()

len (x)


14994

In [3]:
x = df.customer_id.unique ().tolist()

len (x)

99441

In [20]:
import pandas as pd

df = pd.read_csv('olist_geolocation_dataset.csv', delimiter = ',')
df

,geolocation_zip_code_prefix,geolocation_lat,geolocation_lng,geolocation_city,geolocation_state
0,1037,-23.545621,-46.639292,sao paulo,SP
1,1046,-23.546081,-46.644820,sao paulo,SP
2,1046,-23.546129,-46.642951,sao paulo,SP
3,1041,-23.544392,-46.639499,sao paulo,SP
4,1035,-23.541578,-46.641607,sao paulo,SP
...,...,...,...,...,...
1000158,99950,-28.068639,-52.010705,tapejara,RS
1000159,99900,-27.877125,-52.224882,getulio vargas,RS
1000160,99950,-28.071855,-52.014716,tapejara,RS
1000161,99980,-28.388932,-51.846871,david canabarro,RS


In [4]:
import pandas as pd

df_orders = pd.read_csv('olist_orders_dataset.csv', delimiter = ',')
df_orders

l = df_orders.columns.tolist()
l

['order_id',
 'customer_id',
 'order_status',
 'order_purchase_timestamp',
 'order_approved_at',
 'order_delivered_carrier_date',
 'order_delivered_customer_date',
 'order_estimated_delivery_date']

In [11]:
x = df_orders.customer_id.unique ().tolist()

len (x)

99441

In [18]:
import pandas as pd

df_items = pd.read_csv('olist_order_items_dataset.csv', delimiter = ',')
df_items

x = df_items.columns.tolist()
x

['order_id',
 'order_item_id',
 'product_id',
 'seller_id',
 'shipping_limit_date',
 'price',
 'freight_value']

In [14]:
x = df_items.order_id.unique ().tolist()

len (x)

98666

In [54]:
import pandas as pd

df_products = pd.read_csv('olist_products_dataset.csv', delimiter = ',')
df_products

x = df_products.columns.tolist()
x

['product_id',
 'product_category_name',
 'product_name_lenght',
 'product_description_lenght',
 'product_photos_qty',
 'product_weight_g',
 'product_length_cm',
 'product_height_cm',
 'product_width_cm']

In [58]:
x = df_products.product_id.unique ().tolist()

len (x)



32951

In [59]:
import pandas as pd

df_translation = pd.read_csv('product_category_name_translation.csv', delimiter = ',')


x = df_translation.columns.tolist()
x

['product_category_name', 'product_category_name_english']

In [63]:
df_translation.size

142

In [76]:

        
L = Lecture_csv ('./', 'product_category_name_translation.csv')
dico = {}

for row in L.read_iterator ():
    try:
        dico [row['\ufeffproduct_category_name'] ] = row
    except:
        print (row)
        raise
    continue

In [79]:
import pandas as pd

df_sellers = pd.read_csv('olist_sellers_dataset.csv', delimiter = ',')


x = df_sellers.columns.tolist()
x

['seller_id', 'seller_zip_code_prefix', 'seller_city', 'seller_state']

In [80]:
df_sellers.size

12380

In [91]:

import pandas as pd

df_payment = pd.read_csv('olist_order_payments_dataset.csv', delimiter = ',')


x = df_payment.columns.tolist()
x

['order_id',
 'payment_sequential',
 'payment_type',
 'payment_installments',
 'payment_value']

In [102]:


import pandas as pd

df_review = pd.read_csv('olist_order_reviews_dataset.csv', delimiter = ',')


x = df_review.columns.tolist()
x

['review_id',
 'order_id',
 'review_score',
 'review_comment_title',
 'review_comment_message',
 'review_creation_date',
 'review_answer_timestamp']

In [104]:
df_review.size

700000

In [108]:
L = Lecture_csv ('./', 'olist_order_reviews_dataset.csv')


for row in L.read_iterator ():
    x = row['review_score']
    
    continue